In [1]:
from rdflib import Namespace
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, TemplateLibrary

We need a namespace (`BLDG`) to name the entities that will exist in our model. `NUM_VAVs` will be used to generate a number of VAVs for this example

In [2]:
NUM_VAVs = 3
BLDG = Namespace("urn:my-building/")

Create an in-memory BuildingMOTIF instance and create a `my-building` model that we will populate by evaluating templates

In [3]:
bm = BuildingMOTIF("sqlite://")
bldg = Model.create("my-building")

Load in templates / classes from the Brick ontology (this may take 1-2 minutes):

In [4]:
TemplateLibrary.load(ontology_graph="../libraries/brick/Brick-subset.ttl")

TemplateLibrary(_id=1, _name=rdflib.term.URIRef('https://brickschema.org/schema/1.3-subset/Brick'), _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x7f91e8071280>)

We load the G36 template library and pull out the "vav-cooling-only" template, corresponding to the pointlist in Section 4.1 of Guideline 36

In [5]:
lib = TemplateLibrary.load(directory="../libraries/ashrae/guideline36/")
vav_templ = lib.get_template_by_name("vav-cooling-only")

Now that we have the template, we do the following to create the VAV:

In [6]:
# create a "name" for the VAV
vav_name = BLDG["vav-0"]
# evaluate the template with that name to make sure the VAV has that name
tmp = vav_templ.evaluate({"name": vav_name})
# check what parameters are left
print(tmp.parameters)

{'dmp', 'dat', 'co2', 'zone', 'occ', 'ztemp'}


If we don't want to provide names for these yet, we can call `.fill()` to invent names for them. This is helpful for testing and prototyping

In [7]:
bindings, vav_graph = tmp.fill(BLDG) # tell 'fill' to put the invented names in the BLDG namespace

The `vav_graph` object can now be added to our model:

In [8]:
bldg.add_graph(vav_graph)

In [9]:
# run for the other n-1 VAVs
for vav in range(1,NUM_VAVs):
    vav_name = BLDG[f"vav-{vav}"]
    _, vav_graph = vav_templ.evaluate({"name": vav_name}).fill(BLDG)
    bldg.add_graph(vav_graph)

Print out the resulting model

In [10]:
print(bldg.graph.serialize(format="turtle"))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/vav-0> a brick:VAV ;
    brick:feeds <urn:my-building/zone_fa151ed4> ;
    brick:hasPart <urn:my-building/dmp_13de9f93> ;
    brick:hasPoint <urn:my-building/co2_01b46f25>,
        <urn:my-building/dat_9a8c8741>,
        <urn:my-building/occ_7bb61c87>,
        <urn:my-building/ztemp_996574f9> .

<urn:my-building/vav-1> a brick:VAV ;
    brick:feeds <urn:my-building/zone_32486d3d> ;
    brick:hasPart <urn:my-building/dmp_7f79a9c4> ;
    brick:hasPoint <urn:my-building/co2_8d797af9>,
        <urn:my-building/dat_6c052ba6>,
        <urn:my-building/occ_8e858f98>,
        <urn:my-building/ztemp_86f75beb> .

<urn:my-building/vav-2> a brick:VAV ;
    brick:feeds <urn:my-building/zone_b6f5a33e> ;
    brick:hasPart <urn:my-building/dmp_7e5a03da> ;
    brick:hasPoint <urn:my-building/co2_d831eb2a>,
        <urn:my-building/dat_b344c9bc>,
        <urn:my-building/occ_27b80795>,
        <urn:my-building/ztemp_1a60fcdd> .



In [11]:
# and save your work!
bm.session.commit()

Rewinding a little bit, you may have noticed that the generated graph above doesn't actually contain any metadata for the points and parts associated with the VAVs. This is because we didn't deal with dependencies at all. We can use `inline_dependencies()` to resolve other definitions required by the VAV template:

In [12]:
# without resolving dependencies
_, g = vav_templ.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/name_86db738b> a brick:VAV ;
    brick:feeds <urn:my-building/zone_beef52f7> ;
    brick:hasPart <urn:my-building/dmp_8f7135ff> ;
    brick:hasPoint <urn:my-building/co2_241f8e39>,
        <urn:my-building/dat_a9744a32>,
        <urn:my-building/occ_86f532d1>,
        <urn:my-building/ztemp_4bebac62> .




In [13]:
# with resolved dependencies
inlined = vav_templ.inline_dependencies()
_, g = inlined.fill(BLDG)
print(g.serialize())

inlined @prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:___param___#co2> a brick:CO2_Level_Sensor .


inlined @prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:___param___#dat> a brick:Discharge_Air_Temperature_Sensor .


inlined @prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:___param___#zone> a brick:HVAC_Zone .


inlined @prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:___param___#ztemp> a brick:Zone_Air_Temperature_Sensor .


inlined @prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:___param___#occ> a brick:Occupancy_Sensor .


inlined @prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:___param___#dmppos> a brick:Damper_Position_Command .


inlined @prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:___param___#dmp> a brick:Damper ;
    brick:hasPoint <urn:___param___#dmppos> .

<urn:___param___#dmppos> a brick:Damper_Position_Command .


Replacing urn:___param___#dmppos with urn:___param__

In [14]:
print(inlined.body.serialize(format='turtle'))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:___param___#name> a brick:VAV ;
    brick:feeds <urn:___param___#zone> ;
    brick:hasPart <urn:___param___#dmp> ;
    brick:hasPoint <urn:___param___#co2>,
        <urn:___param___#dat>,
        <urn:___param___#occ>,
        <urn:___param___#ztemp> .

<urn:___param___#co2> a brick:CO2_Level_Sensor .

<urn:___param___#dat> a brick:Discharge_Air_Temperature_Sensor .

<urn:___param___#dmp> a brick:Damper ;
    brick:hasPoint <urn:___param___#dmppos-88f0dbb3-inlined> .

<urn:___param___#dmppos-88f0dbb3-inlined> a brick:Damper_Position_Command .

<urn:___param___#occ> a brick:Occupancy_Sensor .

<urn:___param___#zone> a brick:HVAC_Zone .

<urn:___param___#ztemp> a brick:Zone_Air_Temperature_Sensor .


